Goal:
- Extract all audio and create .npy files of batches

In [1]:
import os

import numpy as np
import torch
import torchaudio

from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
# Function to turn a directory into single .npy file
def chunk_audio(dir, seconds):
    files = sorted([file for file in os.listdir(dir) if file.endswith(".wav")])
    big_waveform = torch.zeros(0)
    for file in tqdm(files):
        waveform, sample_rate = torchaudio.load(f"{dir}/{file}")
        chunk_size = int(sample_rate * seconds)

        assert sample_rate == 24000  # Verify sampling rate of 24 kHz
        big_waveform = torch.cat([big_waveform,\
                                  waveform.flatten()])

        # Grab clips every three seconds
        while len(big_waveform) >= chunk_size:
            yield big_waveform[:chunk_size]
            big_waveform = big_waveform[chunk_size:]

In [5]:
root_dir = "./data/LibriTTS/train-clean-100"
dirs = sorted(os.listdir(root_dir))[0:]

for dir in dirs:
    for subdir in sorted(os.listdir(f"{root_dir}/{dir}")):
        # Check if first file in this sequence already exists
        if os.path.exists(f"./data/LibriTTS/train-clean-100-0.2s/{dir:>04}_{subdir:>06}_{0:>05}.npy"):
            continue

        for i, chunk in enumerate(chunk_audio(f"{root_dir}/{dir}/{subdir}", seconds=0.2)):
            np.save(f"./data/LibriTTS/train-clean-100-0.2s/{dir:>04}_{subdir:>06}_{i:>05}.npy", chunk)

  0%|          | 0/66 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:01<00:00,  6.91it/s]
